## Perubahan Struktur Dataset Pembelian dan Stok

### Perubahan Struktur Dataset Pembelian

In [2]:
import pandas as pd
import re
from pathlib import Path

def parse_number(x):
    """Konversi string angka format Indonesia menjadi float"""
    if pd.isna(x) or str(x).strip() == '':
        return 0.0
    x = str(x).replace('.', '').replace(',', '.')
    try:
        return float(x)
    except:
        return 0.0

# Lokasi file
file_path = Path("pembelian.tsv")

# Baca file baris per baris
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []
kode, nama, unit = '', '', ''

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Deteksi baris header produk (contoh: A000001 ANATON TAB STRIP)
    header_match = re.match(r'^([A-Z0-9]+)\s+(.+?)\s+(STRIP|BTL|BOX|PCS)$', line)
    if header_match:
        kode = header_match.group(1).strip()
        nama = header_match.group(2).strip()
        unit = header_match.group(3).strip()
        continue

    # Deteksi baris transaksi (dimulai tanggal DD-MM-YY)
    date_match = re.match(r'^(\d{2}-\d{2}-\d{2})', line)
    if date_match:
        parts = re.split(r'\s{2,}', line)
        tanggal = parts[0]
        no_transaksi = parts[1] if len(parts) > 1 else ''

        qty_msk = nilai_msk = qty_klr = nilai_klr = 0.0

        # Format umum: tanggal | no_transaksi | qty | nilai
        if len(parts) == 4:
            # Cek kode transaksi (1. = masuk, 2. = keluar)
            if no_transaksi.strip().startswith('1.'):
                qty_msk = parse_number(parts[2])
                nilai_msk = parse_number(parts[3])
            elif no_transaksi.strip().startswith('2.'):
                qty_klr = parse_number(parts[2])
                nilai_klr = parse_number(parts[3])

        elif len(parts) == 5:
            # Bisa terjadi karena format tidak konsisten, kita cek pola juga
            if no_transaksi.strip().startswith('1.'):
                qty_msk = parse_number(parts[2])
                nilai_msk = parse_number(parts[3])
            elif no_transaksi.strip().startswith('2.'):
                qty_klr = parse_number(parts[2])
                nilai_klr = parse_number(parts[3])

        elif len(parts) >= 6:
            # Kalau keduanya ada (jarang terjadi)
            qty_msk = parse_number(parts[2])
            nilai_msk = parse_number(parts[3])
            qty_klr = parse_number(parts[4])
            nilai_klr = parse_number(parts[5])

        # Tentukan kategori dan total
        if qty_msk > 0:
            kategori = 'MASUK'
            qty_total = qty_msk
            nilai_total = nilai_msk
        elif qty_klr > 0:
            kategori = 'KELUAR'
            qty_total = qty_klr
            nilai_total = nilai_klr
        else:
            kategori = 'LAINNYA'
            qty_total = 0
            nilai_total = 0

        data.append({
            'KODE': kode,
            'NAMA_PRODUK': nama,
            'UNIT': unit,
            'TANGGAL': pd.to_datetime(tanggal, format='%d-%m-%y', errors='coerce'),
            'NO_TRANSAKSI': no_transaksi,
            'QTY_MSK': qty_msk,
            'NILAI_MSK': nilai_msk,
            'QTY_KLR': qty_klr,
            'NILAI_KLR': nilai_klr,
            'KATEGORI': kategori,
            'QTY_TOTAL': qty_total,
            'NILAI_TOTAL': nilai_total
        })

# Buat DataFrame
df = pd.DataFrame(data)

# Urutkan berdasarkan kode dan tanggal
df = df.sort_values(by=["KODE", "TANGGAL"]).reset_index(drop=True)

# Simpan hasil ke CSV
output_path = "pembelian_final_fix.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ File '{output_path}' berhasil dibuat!")
print(df.head(15))


✅ File 'pembelian_final_fix.csv' berhasil dibuat!
       KODE NAMA_PRODUK   UNIT    TANGGAL          NO_TRANSAKSI  QTY_MSK  \
0   A000001  ANATON TAB  STRIP 2021-07-06  1.13-210706.0908-003     10.0   
1   A000001  ANATON TAB  STRIP 2021-07-12   2.6-210712.1519-097      0.0   
2   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1633-013      0.0   
3   A000001  ANATON TAB  STRIP 2021-07-12  2.13-210712.1807-013      0.0   
4   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1855-018      0.0   
5   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1925-027      0.0   
6   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1957-035      0.0   
7   A000001  ANATON TAB  STRIP 2021-07-12   2.6-210712.0907-023      0.0   
8   A000001  ANATON TAB  STRIP 2021-07-13  2.11-210713.1102-011      0.0   
9   A000001  ANATON TAB  STRIP 2021-07-13   2.6-210713.1701-006      0.0   
10  A000001  ANATON TAB  STRIP 2021-08-23  1.12-210823.1955-003     10.0   
11  A000001  ANATON TAB  STRIP 2021-09

### Perubahan Struktur Dataset Stok

In [3]:
import pandas as pd
import re
from pathlib import Path

# === Fungsi bantu untuk parsing angka format Indonesia ===
def parse_number(x):
    """Konversi string angka format Indonesia menjadi float"""
    if pd.isna(x) or str(x).strip() == '':
        return 0.0
    x = str(x).replace('.', '').replace(',', '.')
    try:
        return float(x)
    except:
        return 0.0

# === Lokasi file input ===
file_path = Path("stok.tsv")

# Baca file baris per baris
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Lewati baris header
    if line.startswith("KODE") or line.startswith("Kode"):
        continue

    # Pisahkan berdasarkan 2+ spasi
    parts = re.split(r'\s{2,}', line)
    # Contoh hasil split:
    # ['A000001', 'ANATON TAB', 'ETL1', '12,00', 'STRIP']

    if len(parts) >= 5:
        kode = parts[0].strip()
        nama_produk = parts[1].strip()
        lokasi = parts[2].strip()
        qty_stok = parse_number(parts[3])
        unit = parts[4].strip()

        data.append({
            'KODE': kode,
            'NAMA_PRODUK': nama_produk,
            'LOKASI': lokasi,
            'QTY_STOK': qty_stok,
            'UNIT': unit
        })

# Buat DataFrame
df = pd.DataFrame(data)

# Urutkan berdasarkan KODE
df = df.sort_values(by=["KODE"]).reset_index(drop=True)

# Simpan ke CSV
output_path = "stok_final_fix.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ File '{output_path}' berhasil dibuat!")
print(df.head(10))


✅ File 'stok_final_fix.csv' berhasil dibuat!
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT
0  A000001          ANATON TAB   ETL1      12.0  STRIP
1   A00001       ACTIVED HIJAU  ETL3A       2.0    BTL
2  A000012  APIALYS SYR 100 ML  ETL3A       2.0    BTL
3  A000014     ALKOHOL 1000 ML  ETL3B       7.0    BTL
4  A000016     ALLOPURINOL 300   RAK2      40.0  STRIP
5  A000018   ATORVASTATIN 10MG   RAK2       6.0  STRIP
6   A00004     ACYCLOVIR 200MG   RAK2      13.0  STRIP
7  A000040         MEFIX 500MG   RAK1       9.0  STRIP
8   A00005     ACYCLOVIR 400MG   RAK2      21.0  STRIP
9  A000066       ANDALAN KB FE   RAK4      35.0  STRIP
